In [5]:
import arcpy
import os

# Set workspace and overwrite
arcpy.env.workspace = r"E:\LandCoverUS\Merged_emp_data\Emp_data.gdb"
arcpy.env.overwriteOutput = True

# Output summary tables
output_tables = [
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_emp",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_emp_1",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_emp_2",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_emp_4"
]

# Corresponding buffer layers
buffer_layers = [
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe1",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe2",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe4"
]

# List all feature classes in the workspace
fc_list = arcpy.ListFeatureClasses()

# Function to run spatial join and insert summarized data
def run_spatial_join(buffer_layer, output_table, suffix):
    for fc in fc_list:
        year = fc.split("_")[-1]
        joined_output = os.path.join(arcpy.env.workspace, f"BufEmp_{suffix}_{year}")

        # Delete if already exists
        if arcpy.Exists(joined_output):
            arcpy.management.Delete(joined_output)

        # Spatial Join
        arcpy.analysis.SpatialJoin(
            target_features=buffer_layer,
            join_features=fc,
            out_feature_class=joined_output,
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            match_option="INTERSECT"
        )

        # Add and calculate employment count field
        arcpy.AddField_management(joined_output, "BlockEmp", "LONG")
        arcpy.CalculateField_management(joined_output, "BlockEmp", "!Join_Count!", "PYTHON3")

        # Transfer to output table
        fields = ["TN_TIMES_I", "BlockEmp", "Area"]
        with arcpy.da.SearchCursor(joined_output, fields) as cursor, \
             arcpy.da.InsertCursor(output_table, ["TN_TIMES_I", "Year", "BlockEmp", "Area"]) as insert_cursor:
            for row in cursor:
                insert_cursor.insertRow((row[0], int(year), row[1], row[2]))

# Run for each buffer and output table
for i in range(4):
    run_spatial_join(buffer_layers[i], output_tables[i], suffix=f"buf{i+1}")
    print(f"Done with buffer {i+1}")


Done with buffer 1
Done with buffer 2
Done with buffer 3
Done with buffer 4


In [6]:
# Set workspace and overwrite
arcpy.env.workspace = r"E:\LandCoverUS\PopulationMerged"  # Update as needed
arcpy.env.overwriteOutput = True

# Output summary tables
output_tables = [
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_pop",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_pop_1",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_pop_2",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\study_pop_4"
]

# Corresponding buffer layers
buffer_layers = [
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe1",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe2",
    r"C:\Users\USER\OneDrive - The University of Memphis\TDOT_project\TDOT\TDOT.gdb\StuydSites_Apr_PairwiseBuffe4"
]

# List all population feature classes
fc_list = arcpy.ListFeatureClasses()

# Function to run spatial join and insert summarized data
def run_population_join(buffer_layer, output_table, suffix):
    for fc in fc_list:
        year1 = fc.split("_")[-1]
        year = year1.split(".")[0]
        joined_output = os.path.join(arcpy.env.workspace, f"BufPop_{suffix}_{year}")

        # Delete if already exists
        if arcpy.Exists(joined_output):
            arcpy.management.Delete(joined_output)

        # Spatial Join
        arcpy.analysis.SpatialJoin(
            target_features=buffer_layer,
            join_features=fc,
            out_feature_class=joined_output,
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            match_option="INTERSECT"
        )

        # Add and calculate population field
        arcpy.AddField_management(joined_output, "BlockPop", "LONG")
        arcpy.CalculateField_management(joined_output, "BlockPop", "!Join_Count!", "PYTHON3")

        # Transfer to output table
        fields = ["TN_TIMES_I", "BlockPop", "Area"]
        with arcpy.da.SearchCursor(joined_output, fields) as cursor, \
             arcpy.da.InsertCursor(output_table, ["TN_TIMES_I", "Year", "BlockPop", "Area"]) as insert_cursor:
            for row in cursor:
                insert_cursor.insertRow((row[0], int(year), row[1], row[2]))

# Run for each buffer and output table
for i in range(4):
    run_population_join(buffer_layers[i], output_tables[i], suffix=f"buf{i+1}")
    print(f"Done with population buffer {i+1}")


Done with population buffer 1
Done with population buffer 2
Done with population buffer 3
Done with population buffer 4
